In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_rl
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
import os
import warnings
from dataclasses import dataclass
from typing import Protocol

import casadi
import do_mpc
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from gymnasium import Env
from IPython.display import HTML
from ipywidgets import interact, widgets
from matplotlib.animation import FuncAnimation
from numpy.typing import NDArray

from training_rl.control import (
    create_inverted_pendulum_environment,
    create_mass_spring_damper_environment,
    MassSpringDamperParameters,
    InvertedPendulumParameters,
    plot_inverted_pendulum_results,
    create_shortest_path_graph,
    plot_shortest_path_graph,
    plot_all_paths_graph,
    animate_mass_spring_damper_simulation,
    animate_inverted_pendulum_simulation,
    animate_full_inverted_pendulum_simulation,
    display_array,
    show_video,
)

warnings.simplefilter("ignore", UserWarning)
sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]
# This is needed because inside docker the rendering of mujoco environments doesn't work.
render_mode = "rgb_array" if os.getlogin() != "jovyan" else None

<img src="_static/images/aai-institute-cover.svg" alt="presentation first slide" style="width:100%;">
<div class="md-slide title">Control and Planning</div>

# Introduction

In previous sections, we have designed feedback controllers for various systems with the goal of regulating the system output to a desired setpoint. Specifically, we utilized Fullstate Feedback and PID controllers. While these simple controllers can effectively regulate many systems, they have limitations that prevent high performance control for more complex systems.

First, these controllers have a fixed, static gain that does not change over time. This limits their ability to adapt to changing system dynamics or disturbances. Second, after developing a mathematical model of the system dynamics, we did not actually utilize the model in the controller design. A model-based approach could allow us to leverage our understanding of the system to improve control performance.

One ubiquitous challenge in control system design is the presence of control input constraints. Actuators in physical systems inherently have limits on their amplitude and rate of change. For example, a motor has maximum torque and acceleration limits. If control input constraints are ignored in the controller design, the resulting control inputs may saturate the actuators, degrading closed-loop performance.

There are two main strategies to address control input constraints:

- Reduce the performance requirements to levels achievable with a linear controller without violating the constraints.
- Directly account for the constraints by modifying the control design and online optimization of the control input.   For example, model predictive control utilizes the system model to predict future behavior and optimize the control input while satisfying constraints.

In subsequent sections, we will explore model-based and optimization-based control techniques to overcome the limitations of basic controllers. Properly managing control input constraints is crucial to achieving high performance in real control systems.

<div>
<figure style="float: left; width: 60%;">
    <img src="_static/images/30_constrained_motion.png" width="100%"/>
</figure>
<div style="float: right; width: 39%;">
<br><br>Illustration of constrained motion of a car from point A to point B. There are state (position/velocity) constraints, and control (acceleration) constraints. When there are mobile obstacles, the state constraints may change unpredictably, necessitating on-line replanning.
</div>
</div>

In this part of the training, we will move away from classical control methods and focus on optimal control methods applied to linear and non-linear systems.

We will use the same 2 systems we have studied previously (Mass-Spring-Damper and Inverted Pendulum). We will additionally consider controlling the full non-linear inverted pendulum (pendulum angle and cart position).

As we present the different methods, we will highlight commonalities and differences with Reinforcement Learning.

For the final exercise we will attempt to control the highly non-linear inverted pendulum system with different objectives.

Finally, we will present AlphaZero and highlight 

# Planning

Planning refers to the explicit deliberation process that chooses and organizes actions by anticipating their
outcomes with the goal to achieve some pre-stated objectives.

Automated planning and scheduling, sometimes denoted as simply AI planning, is a branch of artificial intelligence that concerns the realization of strategies or action sequences, typically for execution by intelligent agents, autonomous robots and unmanned vehicles. Unlike classical control problems, the solutions are complex and must be discovered and optimized in multidimensional space.

In planning we use a model of the environment/system to predict the effect of taking a certain action in a certain state. Thus it is a **model-based** approach.

### State-Transition Systems

A state-transition system is a 3-tuple $\Sigma = (S,A,\gamma)$, where:

- $S = \{s_1,s_2,\dots\}$ is a finite or recursively enumerable set of states.
- $U = \{u_1,u_2,\dots\}$ is a finite or recursively enumerable set of actions.
- $\gamma: S\times U \rightarrow 2^S$ is a state transition function.

- if $u \in U$ and $\gamma(s,u) \neq \emptyset$ then $a$ is applicable in $s$.

- applying $u$ in $s$ will take the system to $s^{\prime} \in \gamma(s,u).$

A state-transition system can be represented by a directed labelled graph $G = (N_G,E_G)$ where:

- the nodes correspond to the states in $S$, i.e. $N_G = S.$
- there is an arc from $s \in N_G$ to $s^\prime \in N_G$, i.e.$s \rightarrow s^\prime \in E_G$, with label $u \in U$ if and only if $s^\prime \in \gamma(s,u).$

### Plans

- a plan $\pi$ is a finite sequence of actions, $\pi = \{u_1, \dots, u_n\}$
- a planning problem is a triple $P = (\Sigma, s_0, S_g)$, where $\Sigma$ is a
  state-transition system, $s_0 \in S$ is an initial state, and $S_g \subset S$ is a set of goal states.
- each node is written as a pair $v = (\pi, s)$ where $\pi$ is a plan and $s = \gamma(s_0, \pi)$

<div>
<figure style="float: left; width: 30%;">
    <img src="_static/images/30_planning_and_plan_execution.png" width="100%"/>
</figure>
<div style="float: left; width: 60%;">

- Planner:
  - given: description of an environment/system $\Sigma$, initial state, and objective.
  - generate: plan that achieves objective.
- Controller:
  - given: plan, current state (observation function: $\eta:S \rightarrow O$)
  - generate: action.
- Environment/System:
  - evolves as actions are executed and events occur
</div>
</div>

# Optimal Control

Optimal control theory is a branch of control theory that deals with finding a control for a dynamical system over a period of time such that an objective function is optimized. The fundamental idea in optimal control is to formulate the goal of control as the long-term optimization of a scalar cost function.

<center>
<figure>
    <img src="_static/images/30_optimal_control_problem.png" width="100%"/>
    <figcaption>
        Deterministic N-stage optimal control problem.
    </figcaption>
</figure>
</center>

The optimal control problem is to find a control $u^* \in \mathbf{U}$ which causes the system $\dot{x}(t) = f(x(t), u(t))$ to follow a trajectory $x^* \in \mathbf{X}$ that minimizes the cost (performance measure):


### Continuous-time

$$
\begin{array}\\
\displaystyle  \min_{u} & J(x_0, u) & \text{(cost)}\\
\text{subject to} & \dot{x}(t) = f(x(t), u(t)) & \text{(dynamical feasibility)}\\
& x(T) \in \mathbf{X_T} & \text{(boundary conditions)}\\
& x(t) \in \mathbf{X} , \forall t \in [0, T] & \text{(state constraints)}\\
& u(t) \in \mathbf{U}, \forall t \in [0, T] & \text{(input constraints)}\\
\end{array}
$$

### Discrete-time

$$
\begin{array}\\
\displaystyle  \min_{u} & J(x_0, u) & \text{(cost)}\\
\text{subject to} & x_{t+1} = f(x_t, u_t) & \text{(dynamical feasibility)}\\
& x_N \in \mathbf{X_N} & \text{(boundary conditions)}\\
& x_t \in \mathbf{X} , \forall t \in \{0, 1, \dots , N - 1\} & \text{(state constraints)}\\
& u_t \in \mathbf{U}, \forall t \in \{0, 1, \dots , N - 1\} & \text{(input constraints)}\\
\end{array}
$$

### Finite Horizon 

- Continuous-time:

$$
J_{0}(x_0, u) = g_T(x(T)) + \int \limits_{0}^{T} g(x(t), u(t)) dt
$$

- Discrete-time:

$$
J_0(x_0, u) = g_N(x_N) + \sum \limits_{k = 0}^{N-1} g_k(x_k, u_k)
$$

### Infinite Horizon

- Continuous-time:

$$
J(x_0, u) = \int \limits_{0}^{\infty} g(x(t), u(t)) dt
$$

- Discrete-time:

$$
J(x_0, u) = \sum \limits_{k = 0}^{\infty} g_k(x_k, u_k)
$$

<center>
<figure>
    <img src="_static/images/30_transition_graph_discrete_system.png" width="100%"/>
    <figcaption>
        Transition graph for a deterministic discrete system.
    </figcaption>
</figure>
</center>

This approach is powerful for a number of reasons. First and foremost, it is very general - allowing us to specify the goal of control equally well for fully- or under-actuated, linear or nonlinear, deterministic or stochastic, and continuous or discrete systems. Second, it permits concise descriptions of potentially very complex desired behaviours, specifying the goal of control as a scalar objective (plus a list of constraints).

Optimal control problems solving methods can be classified in three main families:
Dynamic Programming (DP), Indirect Methods based on calculus of variation and Direct Methods.

- DP is helpful where the number of states is limited and the dynamics are known. It divides an optimal control issue into smaller subproblems and recursively solves each.

- Indirect methods rely on Pontryagin’s Minimum Principle (PMP) to derive the necessary conditions
  for optimality. This method uses the Hamiltonian of the system to reduce the global optimal control problem
  to the solution of a system of $2N$ equations given in the form of a two-point boundary value problem (BVP).  
  Problems involving continuous states and control inputs benefit most from it. 

- Direct methods rely on the discretization of the original optimal control problem which is then transcribed to
  a nonlinear programming problem (NLP) solved numerically using a well-established optimisation method.
  
  There are many direct methods. They differ on how the variables (i.e. control and states) are discretised 
  and on how the continuous time dynamics is approximated.
  
  In the case of shooting and multiple shooting the control are parameterised
  with piecewise linear functions and the differential equations
  are solved via numerical integration. These approaches make use of robust
  and available ordinary differential equations solvers
  but need sensitivity analysis to compute the jacobians
  of the continuity and boundary conditions with respect to the initial and intermediate conditions.

  In the case of state and control parameterisation (direct collocation),
  both states and controls are approximated with polynomial functions,
  therefore the continuous time differential equations are converted into algebraic constraints.

<center>
<figure>
    <img src="_static/images/30_optimal_control_methods.svg" width="80%"/>
    <figcaption>
        Classification of different methods to solve optimal control problems and related formulations and
solution algorithms <a id="biral_notes_2016-back" href="#biral_notes_2016">[Biral Notes 2016]</a>.
    </figcaption>
</figure>
</center>

## Choosing the cost function

Choosing a cost function means translating the system's desired physical state into a mathematical formulation.

Examples:

- Minimum-time problems: $J = t_f - t_0$
- Terminal control problems: $J = || x(t_f) - r(t_f) ||^2$
- Minimum control-effort problems: $J = \sum \limits_{k = t_0}^{t_f} |u(t_k)|$

## Questions

- What are possible cost functions for the 2 systems?

# Systems

## Mass-Spring-Damper Model


<div>
<figure style="float: left; width: 50%;">
    <img src="_static/images/20_mass_spring_damper.svg" width="50%"/>
    <figcaption>
        Classic model used for deriving the equations of a mass spring damper model <a href="#wiki_mass_spring_damper"><b id="wiki_mass_spring_damper-back">[Wiki Mass-Spring-Damper, 2023]</b></a>
    </figcaption>
</figure>
<div style="float: left; width: 40%;">
    
- $z(t)$: Distance along the vertical axis from some reference point.
- $m$: Mass of the object.
- $\lambda$: Coefficient of elasticity.
- $l$: Length of spring.
- $k = \frac{\lambda}{l}$
- $c$: Damping coefficient.
- $f(t)$: Force applied on the object.
- $\gamma$: Motor's gear factor.
- $g$: Gravity.
</div>
</div>

The system has linear state-space model with matrices:

$$
A
=
\begin{bmatrix}
0 & 1 \\
-\frac{k}{m} & -\frac{c}{m}\\
\end{bmatrix};
B
=
\begin{bmatrix}
0 \\
\frac{\gamma}{m}\\
\end{bmatrix};
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix};
D = \begin{bmatrix}
0
\end{bmatrix}
$$

#### State-Space Matrices

In [ ]:
msd_parameters = MassSpringDamperParameters()
# Dynamics matrix
A = np.array(
    [
        [0, 1],
        [-msd_parameters.k / msd_parameters.m, -msd_parameters.c / msd_parameters.m],
    ]
)
# Input matrix
B = np.array([[0, msd_parameters.gamma / msd_parameters.m]]).transpose()
# Output matrices
C = np.array([[1, 0], [0, 1]])
D = np.zeros(2)

#### Model, States and Control inputs

In [ ]:
mass_spring_damper = do_mpc.model.LinearModel("continuous")
mass_spring_damper.set_variable(var_type="_x", var_name="position")
mass_spring_damper.set_variable(var_type="_x", var_name="velocity")
mass_spring_damper.set_variable(var_type="_u", var_name="force")
mass_spring_damper.setup(A, B, C, D)

#### Discretization

In [ ]:
env = create_mass_spring_damper_environment()
mass_spring_damper = mass_spring_damper.discretize(env.dt)

## Inverted Pendulum Model


<div>
<figure style="float: left; width: 40%;">
    <img src="_static/images/20_inverted_pendulum.svg" width="50%"/>
    <figcaption>
        Inverted pendulum model <a href="#goodwin_control_2000"><b id="goodwin_control_2000-back">[Goodwin et al., 2000]</b></a>
    </figcaption>
</figure>
<div style="float: left; width: 50%;">
    
- $y(t)$: distance along the horizontal axis from some reference point.
- $\theta(t)$: angle of the pendulum.
- $M$: mass of the cart.
- $m$: mass of the pendulum (assumed to be concentrated at the tip).
- $l$: length of the pendulum.
- $\mu_c$: cart friction coefficient.
- $\mu_p$: pendulum friction coefficient.
- $f(t)$: force applied on the cart.
</div>
</div>

### Linearized Model

The system has the following partial (pendulum angle and angular velocity only) linearized (around $\theta = 0$ and $\dot{\theta} = 0$) state space model:

$$
A = \begin{bmatrix}
0 & 1 \\
\frac{(M + m)g}{Ml} & 0\\
\end{bmatrix};
B = \begin{bmatrix}
0 \\ -\frac{\gamma}{Ml}
\end{bmatrix};
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix};
D = \begin{bmatrix}
0
\end{bmatrix}
$$

#### State-Space Matrices

In [ ]:
env = create_inverted_pendulum_environment()

ip_parameters = InvertedPendulumParameters(
    l=env.model.geom_pos[2, 2],
    m=env.model.body_mass[2],
    M=env.model.body_mass[1],
)
# Dynamics matrix
A = np.array(
    [
        [0, 1],
        [
            (ip_parameters.M + ip_parameters.m)
            * ip_parameters.g
            / (ip_parameters.m * ip_parameters.l),
            0,
        ],
    ]
)
# Input matrix
B = np.array([[0, -ip_parameters.gamma / (ip_parameters.M * ip_parameters.l)]]).transpose()
# Output matrices
C = np.array([[1, 0], [0, 1]])
D = np.zeros(2)

#### Model, States and Control inputs

In [ ]:
inverted_pendulum_lin = do_mpc.model.LinearModel("continuous")
theta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="theta")
dtheta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="dtheta")
inverted_pendulum_lin.set_variable(var_type="_u", var_name="force");

#### Auxiliary Expressions

We also define the pendulum's kinetic and potential energies:

$E_{\text{kinetic}} = \frac{1}{2} m \left( (l x_2 \cos(x_1))^{2} + (l x_2 \sin(x_1))^{2} \right) + \frac{1}{2}J x_2^2$
    
$E_{\text{potential}} = m g  l \cos(x_1)$

In [ ]:
# Inertia
J = (ip_parameters.m * ip_parameters.l**2) / 3
# Energies
E_kin = 0.5 * J * dtheta**2 + 0.5 * ip_parameters.m * (
    (ip_parameters.l * dtheta * casadi.cos(theta)) ** 2
    + (ip_parameters.l * dtheta * casadi.sin(theta)) ** 2
)
E_pot = ip_parameters.m * ip_parameters.g * ip_parameters.l * casadi.cos(theta)
inverted_pendulum_lin.set_expression("E_kinetic", E_kin)
inverted_pendulum_lin.set_expression("E_potential", E_pot);

#### Model Setup

In [ ]:
inverted_pendulum_lin.setup(A, B, C, D)

#### Discretization

In [ ]:
env = create_inverted_pendulum_environment()
inverted_pendulum_lin = inverted_pendulum_lin.discretize(env.dt)

### Non-Linear Model

We also define the full non-linear model

Application of Newtonian physics to this system leads to the [following model](https://sharpneat.sourceforge.io/research/cart-pole/cart-pole-equations.html):

$$
\ddot{y}(t) = \frac{1}{m\cos^2\theta(t) - (1+k)(M+m)} \left[
mg\sin\theta(t)\cos\theta(t) - (1+k)(\gamma f(t) + ml\dot\theta^2(t) \sin\theta(t) - \mu_c \dot{y}(t)) - \cfrac{\mu_p\cos\theta(t)}{l}\dot{\theta}(t)
\right]\\
\ddot\theta(t) = \frac{1}{(1+k)(M+m)l - ml\cos^2\theta(t)} \left[
(M+m)g\sin\theta(t) - \cos\theta(t) (\gamma f(t) + ml\dot\theta^2(t)\sin\theta(t) - \mu_c \dot{y}(t)) - \cfrac{(M+m)\mu_p}{ml} \dot{\theta}(t)
\right]
$$

with $k = \frac{1}{3}.$

We can convert this to state-space form with input $u(t) = f(t)$ and output
$y(t)$; by introducing:

$$
X(t) = \begin{bmatrix}
x_1(t) \\ x_2(t) \\ x_3(t) \\ x_4(t)
\end{bmatrix}
= \begin{bmatrix}
y(t) \\ \theta(t) \\ \dot{y}(t) \\ \dot{\theta}(t) 
\end{bmatrix}
$$

The system has the following full state-space model:

$$
\dot{X}(t) = \begin{bmatrix}
\dot{x_1}(t) \\ \dot{x_2}(t) \\ \dot{x_3}(t) \\ \dot{x_4}(t)
\end{bmatrix} =
\begin{bmatrix}
x_2(t) \\
x_4(t) \\
\frac{1}{m\cos^2 x_2(t) - (1+k)(M+m)} \left[
mg\sin x_2(t)\cos x_2(t) - (1+k)(\gamma u(t) + ml x_4^2(t) \sin x_2(t) - \mu_c x_3(t)) - \cfrac{\mu_p \cos x_2(t)}{l} x_4(t)
\right]\\
\frac{1}{(1+k)(M+m)l - ml\cos^2 x_2(t)} \left[
(M+m)g\sin x_2(t) - \cos x_2(t) (\gamma u(t) + ml x_4^2(t)\sin x_2(t) - \mu_c x_3(t)) - \cfrac{(M+m)\mu_p}{ml} x_4(t)
\right]
\end{bmatrix}
$$

#### Model, States and Control inputs

In [ ]:
inverted_pendulum = do_mpc.model.Model("continuous")

pos = inverted_pendulum.set_variable(var_type="_x", var_name="position")
theta = inverted_pendulum.set_variable(var_type="_x", var_name="theta")
dpos = inverted_pendulum.set_variable(var_type="_x", var_name="velocity")
dtheta = inverted_pendulum.set_variable(var_type="_x", var_name="dtheta")
u = inverted_pendulum.set_variable(var_type="_u", var_name="force")

#### ODE

In [ ]:
k = 1 / 3
numerator = (
    (ip_parameters.M + ip_parameters.m) * ip_parameters.g * casadi.sin(theta)
    - casadi.cos(theta) * (
        ip_parameters.gamma * u 
        + ip_parameters.m * ip_parameters.l * dtheta**2 * casadi.sin(theta)
        - ip_parameters.mu_c * dpos
    )
    - (ip_parameters.M + ip_parameters.m) / (ip_parameters.m * ip_parameters.l) * ip_parameters.mu_p * dtheta
)
denominator = (
    (1+k)*(ip_parameters.M + ip_parameters.m) * ip_parameters.l
    - ip_parameters.m * ip_parameters.l * casadi.cos(theta) ** 2
)
ddtheta = numerator / denominator

numerator = (
    ip_parameters.m * ip_parameters.g * casadi.cos(theta) * casadi.sin(theta)
    - (1+k)*(
        ip_parameters.gamma * u
        + ip_parameters.m * ip_parameters.l * dtheta**2 * casadi.sin(theta)
        - ip_parameters.mu_c * dpos
    )
    - ip_parameters.mu_p * dtheta * casadi.cos(theta) / ip_parameters.l
)
denominator = ip_parameters.m * casadi.cos(theta)**2 - (1+k)*(ip_parameters.M + ip_parameters.m)
ddpos = numerator / denominator

In [ ]:
inverted_pendulum.set_rhs("position", dpos)
inverted_pendulum.set_rhs("theta", dtheta)
inverted_pendulum.set_rhs("velocity", ddpos)
inverted_pendulum.set_rhs("dtheta", ddtheta)

#### Auxiliary Expressions

And kinetic and potential energies:

$E_{\text{kinetic}} = E_{\text{kinetic, cart}} + E_{\text{kinetic, pendulum}}$
    
$E_{\text{kinetic, cart}} = \frac{1}{2} M x_3^{2}$
    
$E_{\text{kinetic, pendulum}} = \frac{1}{2} m \left( (x_3 + l x_4 \cos(x_2))^{2} + (l x_4 \sin(x_2))^{2} \right) + \frac{1}{2} J x_4$
    
$E_{\text{potential}} = m g  l \cos(x_2)$

In [ ]:
# Inertia
J = (ip_parameters.m * ip_parameters.l**2) / 3
# Energies
E_kin = (
    0.5 * J * dtheta**2
    + 0.5 * ip_parameters.M * dpos**2
    + 0.5
    * ip_parameters.m
    * (
        (dpos + ip_parameters.l * dtheta * casadi.cos(theta)) ** 2
        + (ip_parameters.l * dtheta * casadi.sin(theta)) ** 2
    )
)
E_pot = ip_parameters.m * ip_parameters.g * ip_parameters.l * casadi.cos(theta)
inverted_pendulum.set_expression("E_kinetic", E_kin)
inverted_pendulum.set_expression("E_potential", E_pot);

#### Model Setup

In [ ]:
inverted_pendulum.setup()

## Helpers

In [ ]:
class FeedbackController(Protocol):
    def act(self, observation: NDArray) -> NDArray:
        ...


class ConstantController:
    def __init__(self, u: float) -> None:
        self.u = np.asarray([u])

    def act(self, observation: NDArray) -> NDArray:
        return self.u

In [ ]:
@dataclass
class EnvironmentSimulationResults:
    frames: list[NDArray]
    observations: NDArray
    actions: NDArray

In [ ]:
def simulate_environment(
    env: Env,
    *,
    controller: FeedbackController,
    max_steps: int = 500,
) -> EnvironmentSimulationResults:
    if controller is None:
        controller = RandomController(env)

    observation, _ = env.reset()
    actions = []
    observations = [observation]
    frames = []

    for _ in range(max_steps):
        action = controller.act(observation)
        observation, _, terminated, truncated, _ = env.step(action)

        observations.append(observation)
        actions.append(action)

        # Check if we need to stop the simulation
        if terminated or truncated:
            if env.render_mode is not None:
                frames = env.render()
            env.reset()
            break
    env.close()

    actions = np.stack(actions)
    observations = np.stack(observations)

    return EnvironmentSimulationResults(
        frames=frames,
        observations=observations,
        actions=actions,
    )

### Simulators

In [ ]:
env = create_mass_spring_damper_environment()
mass_spring_damper_simulator = do_mpc.simulator.Simulator(mass_spring_damper)
mass_spring_damper_simulator.set_param(t_step=env.dt)
mass_spring_damper_simulator.setup()

In [ ]:
env = create_inverted_pendulum_environment()
inverted_pendulum_lin_simulator = do_mpc.simulator.Simulator(inverted_pendulum_lin)
inverted_pendulum_lin_simulator.set_param(t_step=env.dt)
inverted_pendulum_lin_simulator.setup()

In [ ]:
env = create_inverted_pendulum_environment(cutoff_angle=np.inf)
inverted_pendulum_simulator = do_mpc.simulator.Simulator(inverted_pendulum)
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    "integration_tool": "idas",
    "abstol": 1e-8,
    "reltol": 1e-8,
    "t_step": env.dt,
}
inverted_pendulum_simulator.set_param(**params_simulator)
inverted_pendulum_simulator.setup()

## Dynamic Programming

Dynamic programming (DP) is a method that in general solves optimization problems that involve making a sequence of decisions by determining, for each decision, subproblems that can be solved in like fashion, such that an optimal
solution of the original problem can be found from optimal solutions of sub-
problems. This method is based on Bellman’s Principle of Optimality, which
he phrased as follows:

> An optimal policy has the property that whatever the initial state and
initial decision are, the remaining decisions must constitute an optimal
policy with regard to the state resulting from the first decision.

<figure>
    <img src="_static/images/30_optimality_principle.png"/>
</figure>

Schematic illustration of the principle of optimality. The tail $\{u_k^∗, \dots, u_{N-1}^*\}$ of an optimal sequence $\{u_0^∗, \dots, u_{N-1}^*\}$ is optimal for the tail subproblem that starts at the state $x_k^*$ of the optimal state trajectory.

Dynamic Programming is a very general solution method for problems which have two properties:

- Optimal substructure (Principle of optimality applies)
  - Optimal solution can be decomposed into subproblems, e.g., shortest path
- Overlapping subproblems
  - Subproblems recur many times
  - Solutions can be cached and reused

Dynamic programming is used across a wide variety of domains, e.g.

- Scheduling algorithms
- Graph algorithms (e.g., shortest path algorithms)
- Graphical models in ML (e.g., Viterbi algorithm)
- Bioinformatics (e.g., Sequence alignment, Protein folding) 

We define the **cost-to-go function** (also known as **optimal value function**) as (for the sake of simplicity we will focus on the discrete-time case):

$$
V_0(x_0) := \min_{u \in \mathbf{U}} J_0(x_0, u)
$$

An admissible control sequence $u^*$ is called optimal, if

$$
V_0(x_0) = J_0(x_0, u^*)
$$

For any feasible $x_0 \in \mathbf{X}$ the optimal value function satisfies

$$
V_k(x_0) = \displaystyle \min_{u \in \mathbf{U}} g_{k}(x_0, u) + V_{k+1}(f(x_0, u))
$$

Moreover, if $u^*$ is an optimal control, then

$$
V_0(x_0) = g_{k}(x_0, u) + V_{k+1}(f(x_0, u))
$$

and

$$
V_0(x_0) = J_0(x_0, u^*)
$$

###  DP Algorithm

For every initial state $x_0$, the optimal cost is equal to $V_N(x_0)$, given by the last step of the following algorithm, which proceeds backward in time from stage $N-1$ to stage $0$:

- Start with $V_N(x_N) = g_N(x_N)$
- then for $k = \{0, \dots , N - 1\}$, let:
 
  $$
  V_{k}(x_{k}) = \displaystyle \min_{u \in \mathbf{U}} \left\{ g_{k}(x_{k}, u_{k}) + V_{k+1}(f(x_{k}, u_{k})) \right\}
  $$
  
Once the functions $V_0, \dots , V_N$ have been obtained, we can use a forward algorithm to construct an optimal control sequence $\{u_0^*, \dots, u_{N-1}^*\}$ and corresponding state trajectory $\{x_1^∗, \dots, x_{N}^*\}$ for the given initial state $x_0$ .


<figure>
    <img src="_static/images/30_dynamic_programming.png" width="%80"/>
</figure>

Illustration of the DP algorithm. The tail subproblem that starts at $x_k$ at time $k$ minimizes over
$\{u_k , \dots , u_{N-1}\}$ the "cost-to-go" from $k$ to $N$.

## Exercise

In [ ]:
G = create_shortest_path_graph()
plot_shortest_path_graph(G)

We wish to travel from node A to node G at minimum cost. If the cost represents time then we want to find the shortest path from A to G.

- Arrows (edges) indicate the possible movements.
- Numbers on edges indicate the cost of moving along an edge.

Use Dynamic Programming to solve this problem.

> **Hint** Determine all possible paths first and then compute the cost-to-go at each node.

## Solution

In [ ]:
plot_all_paths_graph(G)

Each node in this new graph represents a state. We will start from the tail (the last states) and compute recursively the cost for each state transition.

We start by determining the cost-to-go for all positions.

Let $l(n_1, n_2)$ the cost of going from node $n_1$ to $n_2$ and $V(n)$ be the cost-to-go from node $n$.

$$
\begin{array}\\
V(\text{ABDF}) &= g(\text{ABDF}, \text{ABDFG}) &= 1\\
V(\text{ABE}) &= g(\text{ABE}, \text{ABEG}) &= 4\\
V(\text{ACF}) &= g(\text{ACF}, \text{ACFG}) &= 1\\
V(\text{ADF}) &= g(\text{ADF}, \text{ADFG}) &= 1\\
\end{array}
$$

$$
\begin{array}\\
V(\text{ABD}) &= \min \left[ g(\text{ABD}, \text{ABDG}), g(\text{ABD}, \text{ABDF}) + V(\text{ABDF}) \right]
&= \min \left[ 8, 5 + 1 \right] &= 6
\\
V(\text{AB}) &= \min \left[ g(\text{AB}, \text{ABD}) + V(\text{ABD}), g(\text{AB}, \text{ABE}) + V(\text{ABE}) \right]
&= \min \left[ 9 + 6, 1 + 4 \right] &= 5
\\
V(\text{AC}) &= g(\text{AC}, \text{ACF}) + V(\text{ACF}) &= 2 + 1 &= 3
\\
V(\text{AD}) &= \min \left[ g(\text{AD}, \text{ADF}) + V(\text{ADF}), g(\text{AD}, \text{ADG})) \right]
&= \min \left[ 5 + 1, 8 \right] &= 6
\\
\end{array}
$$

$$
\begin{array}\\
V(\text{A}) &= \min \left[
g(\text{A}, \text{AB}) + V(\text{AB}), g(\text{A}, \text{AC}) + V(\text{AC}), g(\text{A}, \text{AD}) + V(\text{AD})
\right]
&= \min \left[ 1 + 5, 5 + 3, 3 + 6 \right] &= 6
\\
\end{array}
$$

The shortest-path is ABEG.

In [ ]:
plot_all_paths_graph(G, show_solution=True)

## Linear Quadratic Regulator

While solving the optimal control problem (OCP) is very hard in general, there are a few very important special cases where the solutions are very accessible. Most of these involve variants on the case of linear dynamics and quadratic cost. The simplest case, called the linear quadratic regulator (LQR), is formulated as stabilizing a time-invariant linear system to the origin can be solved analytically.

For a discrete-time linear system described by:

$$
\mathbf{x}_{t+1} = A \mathbf{x}_t + B \mathbf{u}_t
$$

where $x\in \mathbb {R} ^{n}$ (that is, $x$ is an $n$-dimensional real-valued vector) is the state of the system and $u\in \mathbb {R} ^{m}$ is the control input. Given a quadratic cost function for the system in the infinite-horizon case, defined as:

$$
J(\mathbf{x}_0, \mathbf{u}) = \sum \limits _{k = 0}^{\infty} \mathbf{x}_k^{T}Q \mathbf{x}_k + \mathbf{u}_k^{T} R \mathbf{u}_k
$$

With $Q = Q^T \succeq 0$, $R = R^T \succeq 0$.

In both cases, the control law that minizes the cost is given by:

$$
u_k = -K x_k
$$

With: $K = (R + B^T P B)^{-1} B^T P B$

and $P$ is found by solving the discrete time algebraic Riccati equation (DARE):

$$
Q + A^{T}PA-(A^{T}PB)(R+B^{T}PB)^{-1}(B^{T}PA) = P.
$$

<center>
<figure>
    <img src="_static/images/30_lqr_block_diagram.svg" width="60%"/>
    <figcaption>
        LQR Block Diagram.
    </figcaption>
</figure>
</center>

### Mass-Spring-Damper

Now, we design Linear Quadratic Regulator for the Mass-Spring-Damper model. First, we create an instance of the LQR class.

In [ ]:
lqr = do_mpc.controller.LQR(mass_spring_damper)

We choose an infinite prediction horizon (`n_horizon = None`), the time step `t_step = 0.04s` second (which is the same as the environment's timestep).

In [ ]:
env = create_mass_spring_damper_environment(render_mode=render_mode)
lqr.settings.t_step = env.dt
lqr.settings.n_horizon = None  # infinite horizon

The goal is to drive the Mass-Spring-Damper system to the desired position. For that we define the following objective.

In [ ]:
Q = np.diag([1000, 0])
R = np.diag([0])
display_array("Q", Q)
display_array("R", R)
lqr.set_objective(Q=Q, R=R)

We then complete the LQR setup.

In [ ]:
lqr.setup()

Finally, we set the desired setpoint.

In [ ]:
xss = np.array([0.1, 0.0]).reshape(-1, 1)
lqr.set_setpoint(xss)

#### SImulation

Now, we simulate the closed-loop system for 50 steps:

In [ ]:
lqr.reset_history()
mass_spring_damper_simulator.reset_history()
x0 = np.zeros((2, 1))
mass_spring_damper_simulator.x0 = x0
for _ in range(50):
    u0 = lqr.make_step(x0)
    x0 = mass_spring_damper_simulator.make_step(u0)

In [ ]:
animate_mass_spring_damper_simulation(lqr.data)

#### Evaluation

Finally, we evaluate the controller on the actual environment.

In [ ]:
class LQRController:
    def __init__(self, lqr: do_mpc.controller.LQR) -> None:
        self.lqr = lqr
        self.lqr.reset_history()

    def act(self, observation: NDArray) -> NDArray:
        return lqr.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
max_steps = 100
env = create_mass_spring_damper_environment(render_mode=render_mode, max_steps=max_steps)
controller = LQRController(lqr)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
plt.close()
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data)
ax[0].hlines(xss[0], lqr.data._time[0], lqr.data._time[-1], "r");

## Exercise

- Change the cost matrices of the LQR problem above and try to find the best combinations. 
- Design an LQR controller for the inverted pendulum.

## Solution

### Inverted Pendulum

In [ ]:
lqr = do_mpc.controller.LQR(inverted_pendulum_lin)

In [ ]:
env = create_inverted_pendulum_environment()
lqr.settings.t_step = env.dt
lqr.settings.n_horizon = None  # infinite horizon

In [ ]:
# Setting the objective
Q = np.diag([100, 1])
R = np.diag([10])
display_array("Q", Q)
display_array("R", R)
lqr.set_objective(Q=Q, R=R)

In [ ]:
lqr.setup()

In [ ]:
# Define set point
xss = np.array([0.0, 0.0]).reshape(-1, 1)
lqr.set_setpoint(xss)

#### Simulation

In [ ]:
lqr.reset_history()
inverted_pendulum_lin_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
inverted_pendulum_lin_simulator.x0 = x0
for k in range(50):
    u0 = lqr.make_step(x0)
    x0 = inverted_pendulum_lin_simulator.make_step(u0)

In [ ]:
animate_inverted_pendulum_simulation(lqr.data)

#### Evaluation

In [ ]:
class LQRController:
    def __init__(self, lqr: do_mpc.controller.LQR) -> None:
        self.lqr = lqr
        self.lqr.reset_history()

    def act(self, observation: NDArray) -> NDArray:
        return lqr.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()

In [ ]:
max_steps = 500
env = create_inverted_pendulum_environment(render_mode=render_mode, max_steps=max_steps)
controller = LQRController(lqr)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
plt.close()
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data, aux_list=[])
ax[0].hlines(xss[0], lqr.data._time[0], lqr.data._time[-1], "r");

# Model Predictive Control

Unfortunately, the analytically convenient linear quadratic problem formulations are often not satisfactory. There are two main reasons for this:

- The system may be nonlinear, and it may be inappropriate to use for
  control purposes. Moreover, some of the control variables may be naturally
  discrete, and this is incompatible with the linear system viewpoint.
  
- There may be control and/or state constraints, which are not handled
  adequately through quadratic penalty terms in the cost function. For
  example, the motion of a car may be constrained by the presence of
  obstacles and hardware limitations.
  The solution obtained from a linear quadratic model may not be suitable for such
  a problem, because quadratic penalties treat constraints "softly"
  and may produce trajectories that violate the constraints.

Model Predictive Control (MPC) is a control algorithm based on solving an **on-line** optimal control problem. A receding horizon approach is used which can be summarized in the following steps:

1. At time $t$ and for the current state $x_t$, solve, on-line, an open-loop optimal control problem over some future interval taking account the current and future constraints.
2. Apply the first step in the optimal control sequence.
3. Repeat the procedure at time $t + 1$ using the current state $x_{t + 1}$.

<div>
<figure style="float: left; width: 60%;">
    <img src="_static/images/30_model_predictive_control_horizon.svg" width="100%"/>
</figure>
<div style="float: right; width: 39%;">
<br><br><br>Illustration of the problem solved by MPC at state $x_k$.
We minimize the cost function over the next $l$ stages.
We then apply the control of the optimizing sequence up to the control horizon.
In most cases, the control horizon is set to 1.
</div>
</div>

<center>
<figure>
    <img src="_static/images/30_mpc_block_diagram.svg" width="80%"/>
    <figcaption>
        MPC Block Diagram.
    </figcaption>
</figure>
</center>

A major difference between MPC and finite-state stochastic control problems
that are popular in the RL/artificial intelligence literature is that in MPC
the state and control spaces are continuous/infinite, such as for example
in self-driving cars, the control of aircraft and drones, or the operation of
chemical processes

## Iterative Linear Quadratic Regulator

Iterative Linear Quadratic Regulator (iLQR) is an extension of LQR control to non-linear system with non-linear quadratic costs.

The idea is to approximate the cost and dynamics as quadratic and affine, respectively, then exactly solve the resulting LQR problem.

Given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a first-order Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}\\
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t\\
\delta\mathbf{x}_{t+1} &= A_t\delta\mathbf{x}_t + B_t\delta\mathbf{u}_t
\end{array}
$$

with:

$$
A_t = A(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\\
B_t = B(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\\
$$

Given a general cost function that is not linear-quadratic, we use a second-order Taylor Series Expansion to linearize the dynamics into a form common for optimal control problems:

$$
\begin{array}\\
J_N(\mathbf{x}_0, \mathbf{U}) &=
l_f(\mathbf{x}_N) + \sum \limits_{t=1}^{N-1} l(\mathbf{x}_t, \mathbf{u}_t)\\
&\approx \frac{1}{2} \mathbf{x}^T_N Q \mathbf{x}_N + q_N^T \mathbf{x}_N
+ \sum \limits_{t=1}^{N-1}
\frac{1}{2} \mathbf{x}^T_t Q_t \mathbf{x}_t
+ \frac{1}{2} \mathbf{u}^T_t R_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{x}^T_t H_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{u}^T_t H_t^T \mathbf{x}_t
+ q_t^T \mathbf{x}_t +  + r_t^T \mathbf{u}_t
\end{array}
$$

We define the following variables for convenience:

$$
\begin{array}\\
l_x &:= \frac{\partial l}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t \mathbf{x}_t + q_t
\\
l_u &:= \frac{\partial l}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t \mathbf{u}_t + r_t
\\
l_{xx} &:= \frac{\partial^2 l}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t
\\
l_{uu} &:= \frac{\partial^2 l}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t
\\
l_{xu} &:= \frac{\partial^2 l}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t
\\
l_{ux} &:= \frac{\partial^2 l}{\partial \mathbf{u}\mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t^T
\end{array}
$$

We can apply Bellman's Principle of Optimality to define the optimal cost-to-go:

$$
\begin{array}\\
V_N(\mathbf{x}_N) &= g_f(\mathbf{x}_N)\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u {g(\mathbf{x}_{t}, \mathbf{u}_{t}) + V_{t+1}(f(\mathbf{x}_{t}, \mathbf{u}_{N-t}))}\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u Q_{t}(\mathbf{x}_{t}, \mathbf{u}_{t})
\end{array}\\
$$

The $Q$-function is the discrete-time analogue of the Hamiltonian, sometimes known as the pseudo-Hamiltonian.

We approximate the cost-to-go function as locally quadratic near the nominal trajectory gives us (we drop the time index in some of the equations for the sake of readability):

$$
\delta V(\mathbf{x}) = s^T \delta\mathbf{x} + \frac{1}{2} \delta\mathbf{x}^T S \delta\mathbf{x}
$$

with 
$s = \frac{\partial V}{\partial \mathbf{x}}|_{\mathbf{x}},
S = \frac{\partial^2 V}{\partial \mathbf{x}^2}|_{\mathbf{x}}$

Similarily:

$$
\delta Q(\mathbf{x}, \mathbf{u}) = 
\frac{1}{2}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}^T
\begin{bmatrix} Q_{xx} & Q_{xu} \\ Q_{ux} & Q_{uu} \end{bmatrix}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
+
\begin{bmatrix} Q_{x} \\ Q_{u} \end{bmatrix}^T
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
$$

with:

$$
\begin{array}\\
Q_x &= l_x + s_{t+1} A_t \\
Q_u &= l_u + s_{t+1} B_t \\
Q_{xx} &= l_{xx} + A_t^T S_{t+1} A_t \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t
\end{array}
$$

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}\\
\Delta V &= \frac{1}{2} k^T Q_{uu} k + k^T Q_u\\
s &= Q_x + K^T Q_{uu} k + K^T Q_u + Q_{ux}^T k\\
S &= Q_{xx} + K^T Q_{uu} K + K^T Q_{ux} + Q_{ux}^T K
\end{array}
$$

Once these terms are computed, a forward pass computes a new trajectory:

$$
\begin{array}\\
\hat{x}_0 &= x_0\\
\hat{u}_t &= u_t + k_t + K_t(\hat{x}_t - x_t)\\
\hat{x}_{t+1} &= f(\hat{x}_t, \hat{u}_t)
\end{array}
$$

The basic flow of the algorithm is:

1. Initialize with initial state $\mathbf{x}_0$ and initial control sequence $\mathbf{U} = \{\mathbf{u}_{0}, \mathbf{u}_{1}, \dots, \mathbf{u}_{N-1}\}$.
2. Do a forward pass using the non-linear dynamics, i.e. simulate the system using $(\mathbf{x}_0, \mathbf{U})$ to get the trajectory through state space, $\mathbf{X}$, that results from applying the control sequence $\mathbf{X}$ starting in $\mathbf{x}_0$.
3. Do a backward pass, estimate the value function and dynamics for each $(\mathbf{x}, \mathbf{u})$ in the state-space and control signal trajectories.
4. Calculate an updated control signal $\hat{\mathbf{U}}$ and evaluate cost of trajectory resulting from $(\mathbf{x}_0, \hat{\mathbf{U}})$.
   
   1. If $|(\textrm{cost}(x_0, \hat{\textbf{U}}) - \textrm{cost}(x_0, \textbf{U})| < \textrm{threshold},$ then we've converged and exit.
   2. If $\textrm{cost}(x_0, \hat{\textbf{U}}) < \textrm{cost}(x_0, \textbf{U}),$ then set $\textbf{U} = \hat{\textbf{U}},$ and change the update size to be more aggressive. Go back to step 2.
   3. If $\textrm{cost}(x_0, \hat{\textbf{U}}) \geq \textrm{cost}(x_0, \textbf{U}),$ change the update size to be more modest. Go back to step 3.

## Differential Dynamic Programming

Differential Dynamic Programming (DDP) is almost identical to iLQR but unlike the latter, it expands the system's dynamics to the second order i.e. for a given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a **second-order** Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}\\
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t
+ \frac{1}{2} \left(
\frac{\partial^2 f}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t^2
+ 2 \frac{\partial^2 f}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t\delta\mathbf{u}_t
+ \frac{\partial^2 f}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t^2
\right)
\\
\delta\mathbf{x}_{t+1} &= f_x\delta\mathbf{x}_t + f_u\delta\mathbf{u}_t + \frac{1}{2} \left(
f_{xx} \delta\mathbf{x}_t^2 + 2 f_{xu} \delta\mathbf{x}_t \delta\mathbf{u}_t + f_{uu} \delta\mathbf{u}_t^2
\right)
\end{array}
$$

Which leads to a different expansion of the $Q$-function:

$$
\begin{array}\\
Q_x &= l_x + f_x^T s_{t+1} \\
Q_u &= l_u + f_u^T s_{t+1} \\
Q_{xx} &= l_{xx} + f_x^T S_{t+1} f_x + S_{t+1} \cdot f_{xx} \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t + S_{t+1} \cdot f_{xu} \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t + S_{t+1} \cdot f_{uu} 
\end{array}
$$

The difference with iLQR is that the last term in the last 3 equations is the product of a vector with a tensor. 

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}\\
\Delta V &= -\frac{1}{2} k^T Q_{uu} k\\
s &= Q_x - K^T Q_{uu} k\\
S &= Q_{xx} - K^T Q_{uu} K
\end{array}
$$

### Mass-Spring-Damper

[do-mpc](https://www.do-mpc.com/en/latest/) uses [CasADi](https://web.casadi.org/python-api/) (an open-source tool for nonlinear optimization and algorithmic differentiation) for the modeling part and for the different cost functions. Here's a table of useful operators:

| Operator | Description |Equation |
| --- | --- | --- |
| [casadi.sumsqr(x)](https://web.casadi.org/python-api/#casadi.casadi.sumsqr) | Squared-sum | $\sum x_i^2$ |
| [casadi.norm_2(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_2) | $L_2$-norm | $\sqrt{\sum x_i^2}$ |
| [casadi.norm_1(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_1) | $L_1$-norm | $\sum |x_i|$ |
| [casadi.bilin(A, x)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Quadratic Form | $x^T A x$ |
| [casadi.bilin(A, x, y)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Bilinear Form | $x^T A y$ |

#### Controller

First, we create an instance of the MPC class is generated with the Mass-Spring-Damper prediction model defined above.

In [ ]:
mpc = do_mpc.controller.MPC(mass_spring_damper)

We choose the finite prediction horizon `n_horizon = 20`, the time step `t_step = 0.04s` to be the same as the environment's time step. We also set the parameters of the applied discretization scheme orthogonal collocation.

In [ ]:
env = create_mass_spring_damper_environment()
mpc_params = {
    "n_horizon": 20,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

The control objective is to move the mass to a desired position (`0.1`) and keep it there.

In [ ]:
xss = np.array([0.1, 0.0])
distance_cost = 100 * casadi.norm_2(mass_spring_damper.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

We also restrict the input force.

In [ ]:
force_penalty = 1e-2
mpc.set_rterm(force=force_penalty)

#### Constraints

We apply constraints on the force. In this case, there is only an upper and lower bounds for the force.

In [ ]:
# lower and upper bounds of the input
u_max = 20
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

We can now setup the controller.

In [ ]:
mpc.setup()

#### Simulation

We set the initial state and simulate the closed-loop for 100 steps.

In [ ]:
%%capture
mpc.reset_history()
mass_spring_damper_simulator.reset_history()
x0 = np.zeros((2, 1))
mass_spring_damper_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = mass_spring_damper_simulator.make_step(u0)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

#### Evaluation

Finally, we evaluate the controller on the actual environment.

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 100
env = create_mass_spring_damper_environment(render_mode=render_mode, max_steps=max_steps)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

## Exercise

- Design an MPC controller for the linearized inverted pendulum.
- For each case, try different cost functions:
  - $\sum \theta^2$
  - $\sum |\theta|$
  - $E_{\text{kinetic}} - E_{\text{potential}}$

## Solution

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum_lin)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 50,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
xss = np.array([0.0, 0.0])
distance_cost = casadi.bilin(np.diag([100, 1]), inverted_pendulum_lin.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 1e-4
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
inverted_pendulum_lin_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
inverted_pendulum_lin_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = inverted_pendulum_lin_simulator.make_step(u0)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(render_mode=render_mode, max_steps=max_steps)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

## Exercise

- Design an MPC Controller for the non-linear inverted pendulum system for two different cases:
  1. Cart at origin and upright Pendulm: Set the reference for the cart position to the origin.
  2. Pendulum Swing-up: Set the initial angle to to $-\pi$ i.e. start with the pendulum at the bottom.
  3. Same as 2 but set the reference for the cart position to the origin.
  4. Make the pendulum rotate as fast as possible.
  
> **Note 1** Use the following to create the environment with initial angle set to -np.pi and cutoff angle to np.inf for second > case.
> ```python
> env = create_inverted_pendulum_environment(cutoff_angle=np.inf, initial_angle=-np.pi)
> ```

> **Note 2**: You can access the inverted pendulum's kinetic, respectively potential, energy using
> `inverted_pendulum.aux["E_kinetic"]`, respectively `inverted_pendulum.aux["E_potential"]`

## Solution

### Swing-up

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 100,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
energy_cost = inverted_pendulum.aux["E_kinetic"] - inverted_pendulum.aux["E_potential"]
terminal_cost = energy_cost
stage_cost = energy_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 0.1
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the position
x_max = 1
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
inverted_pendulum_simulator.reset_history()
x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
inverted_pendulum_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = inverted_pendulum_simulator.make_step(u0)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(
    render_mode=render_mode, max_steps=max_steps, cutoff_angle=np.inf, initial_angle=-np.pi
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

# MPC and AlphaZero

AlphaZero is a computer program developed by artificial intelligence research company DeepMind to master the games of chess, shogi and go.

It is built from three core pieces:

- Value Function (Neural Network) to estimate the optimal cost-to-go for any given state.
- Policy (Neural Network) to determine the action to take at a given state.
- Monte Carlo Tree Search to simulate and search for the best plan.

<div>
<figure style="float: left; width: 60%;">
    <img src="_static/images/30_alphazero_online_play.png" width="100%"/>
</figure>
<div style="float: right; width: 30%;">
Illustration of an on-line player such as the one used in AlphaGo,
AlphaZero, and Tesauro’s backgammon program. At a given position,
it generates a lookahead tree of multiple moves up to some depth, then runs
the off-line obtained player for some more moves, and evaluates the effect of the
remaining moves by using the position evaluator of the off-line player.
</div>
</div>

In AlphaZero, the policy and value networks are trained off-line and an approximate version of the fundamental DP algorithm of policy iteration. A separate on-line player is used to select moves, based on multistep lookahead minimization and a terminal position evaluator that was trained using experience with the off-line player.

This approach performs better than using the off-line policy directly because of the long lookahead minimization, which corrects for the inevitable imperfections of the neural network-trained off-line
player, and position evaluator/terminal cost approximation.

In model predictive control (MPC), there is no off-line training and we use the system's model for the on-line rollout. The control interval is equivalent to the number of steps in lookahead minimization, while the prediction interval is equivalent to the total number of steps in lookahead minimization and truncated rollout.

## Monte Carlo Tree Search

Monte Carlo Tree Search (MCTS) is a heuristic search algorithm which uses stochastic simulations for decision processes, most notably those employed in software that plays board games. In that context MCTS is used to solve the game tree.

MCTS was combined with neural networks in 2016 and has been used in multiple board games like Chess, Shogi, Checkers, Backgammon, Contract Bridge, Go, Scrabble, and Clobber as well as in turn-based-strategy video games (such as Total War: Rome II's implementation in the high level campaign AI). 

The method especially took off due its effectiveness in computer Go and is still being used in DeepMind’s AlphaGoZero the most advanced Go AI to date.

MCTS consists of 4 steps that are repeated until some condition is met:

1. **Selection**: The selection phase traverses the tree level by level, each time
   selecting a node based on stored statistics like "number of visits"
   or "total reward". The rule by which the algorithm selects is called the tree policy.
   Selection stops when a node is reached that is not fully explored yet, i.e.
   not all possible moves have been expanded to new nodes yet.
2. **Expansion**: The expansion step consists of adding one or multiple new 
   child nodes to the final selected node.

3. **Rollout**: A rollout is initiated from each of these added nodes.
   Depending on the mean depth of the tree, rollout is done until the end of the game or
   as deep as possible before performing an evaluation. When dealing with
   terminal states in games the evaluation is done based on the win condition
   and usually yields 0 (loss), 0.5 (draw) or 1 (win).
   In the original algorithm the rollout is performed at random, but in general
   the so-called default policy can be enhanced by heuristics. For example,
   in the case of AlphaGoZero the rollout step was completely replaced by a
   neural network evaluation function.
4. **Backup**: The outcome of the rollout step is backed up to the nodes involved 
   in the selection phase by updating their respective statistics

<figure>
    <img src="_static/images/30_monte_carlo_tree_search.svg" width="100%"/>
</figure>

# Summary

- Planning plays an important role in control, decision making and reinforcement learning.
- Optimal Control is the branch of Control Theory concerned with optimizing a cost function through the use of open-loop planning.
- The Linear Quadratic Regulator (LQR) is an optimal feedback control law that optimizes a quadratic cost function for LTI systems.
- Model Predictive Control (MPC) is a control scheme that uses planning in a receding horizon to determine the best action to take at each time step.
- MPC uses the system's model for the rollout and can approached in many different ways such as iLQR and DDP.
- AlphaZero uses off-line learning and on-line planning to tackle different types of games. Unlike MPC it uses neural networks to determine the next best action and to approximate the cost-to-go. 

<img src="_static/images/aai-institute-cover.svg" alt="Snow" style="width:100%;">
<div class="md-slide title">Thank you for the attention!</div>

# References

- [<b id="slotine_applied_1991">[Slotine 1991]</b>](#slotine_applied_1991-back) [Applied nonlinear control.](https://www.academia.edu/download/33582713/Applied_Nonlinear_Control_Slotine.pdf) Slotine, Jean-Jacques E., and Weiping Li. Vol. 199, no. 1. Englewood Cliffs, NJ: Prentice hall. 1991.

- [<b id="grune_nonlinear_2017">[Lars Nonlinear 2017]</b>](#grune_nonlinear_2017-back) [Nonlinear model predictive control.](https://link.springer.com/chapter/10.1007/978-3-319-46024-6_3) Grüne, Lars, Jürgen Pannek, Lars Grüne, and Jürgen Pannek. Springer International Publishing, 2017.

- [<b id="biral_notes_2016">[Biral Notes 2016]</b>](#biral_notes_2016-back) [Notes on numerical methods for solving optimal control problems.](https://www.jstage.jst.go.jp/article/ieejjia/5/2/5_154/_article/-char/ja/) Biral, Francesco, Enrico Bertolazzi, and Paolo Bosetti. IEEJ Journal of Industry Applications 5, no. 2 (2016): 154-166.

- [<b id="bertsekas_lessons_2022">[Bertsektas, Dimitri P, 2022]</b>](#bertsekas_lessons_2022-back) [Lessons from AlphaZero for Optimal, Model Predictive, and Adaptive Control](http://web.mit.edu/dimitrib/www/LessonsfromAlphazero.pdf) - Dimitri P. Bertsektas. 2022.

- [<b id="spencer_optimal_2023">[Spencer et al. 2023]</b>](#spencer_optimal_2023-back) [AA 203: Optimal and Learning-Based Control.](https://stanfordasl.github.io/aa203/sp2223/) Optimal control solution techniques for systems with known and unknown dynamics. 2023.

- [<b id="tedrake_underactuated_2023">[Russ Tedrake, 2023]</b>](#tedrake_underactuated_2023-back) [Underactuated Robotics: Algorithms for Walking, Running, Swimming, Flying, and Manipulation (Course Notes for MIT 6.832)](http://underactuated.mit.edu/index.html) Russ Tedrake. Downloaded on 24.10.2023.